In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from torch import nn
from torchvision import models
import copy

device = torch.device("cuda")
# %env CUBLAS_WORKSPACE_CONFIG=:4096:8
os.environ["TORCH_HOME"] = "."
import sys

# torch.cuda.is_available()

from torchvision import models

############################################
# parameters to be set for encironment
# NOT NEED to modify if you have the setting

data_root_path = "/opt/local/torch/data/tcr-mat-form"  # path to store train data
util_path = "../../.."  # path to store the util package
############################################

# add local dir to sys path
sys.path.insert(0, util_path)  # the util package is supposed to be clone to this path

In [2]:
from util.model.surf.modified_cnn_model import (
    ModifiedPretrainedNet,
    SurfNet256,
)
from typing import Any, Dict, Union
from util.model.surf.dateset import SurfDatasetFromMat
from util.model.surf.pretrained_model import PretrainedModelDb
import torch.optim as optim


def define_surf_model(
    model_name: str,
    model_type: Union[int, str],
    suffix: str,
    pretrain: bool = True,
    set_type: str = "train",
    kwd: Dict[str, Any] = {
        "dropout": 0.2,
        "cnn_feature_ratio": 0.5,
        "data_root_path": "/hy-tmp/",
        "data_csv_filename": "DataNormilized.csv",
        "lr": 0.001,
        "num_params": 3,
        "num_output": 2,
    },
):
    """
    定义和配置一个用于表面模型的深度学习模型。

    参数:
    - model_name: 模型的名称。
    - model_type: 模型的类型，可以是整数或字符串。
    - suffix: 附加在模型名称和类型之后的后缀，用于区分不同的模型配置。
    - pretrain: 是否使用预训练的模型权重，默认为True。
    - kwd: 包含模型训练和配置所需的各种参数的字典。

    返回:
    一个包含模型配置和训练所需信息的字典，包括模型前缀、模型实例、数据集、优化器和损失函数。
    """

    # 初始化预训练模型数据库
    model_info_db = PretrainedModelDb()

    # 从数据库中获取指定模型和类型的信息
    train_model, model_weights, name_first_conv, name_fc = model_info_db.get_info(
        model_name, model_type
    )

    # 构造模型前缀，用于后续的日志记录或模型保存
    prefix = f"{model_name}{model_type}_{suffix}"

    # 创建一个修改过的预训练网络实例
    pnet = ModifiedPretrainedNet(
        pretrained_net=train_model,
        weights=model_weights if pretrain else None,
        name_first_conv=name_first_conv,
        name_fc=name_fc,
    )

    # 创建并配置最终的表面模型
    surf_model = SurfNet256(
        modified_net=pnet,
        num_params=kwd["num_params"],
        num_output=kwd["num_output"],
        dropout=kwd["dropout"],
        cnn_feature_ratio=kwd["cnn_feature_ratio"],
    )

    # 将模型移动到指定的设备上
    surf_model.to(device)

    # 创建并配置数据集
    dset = SurfDatasetFromMat(
        data_csv_filename=os.path.join(
            kwd["data_root_path"], set_type, kwd["data_csv_filename"]
        ),
        surf_data_dir=os.path.join(kwd["data_root_path"], set_type, "Surf"),
        param_start_idx=3,
        param_end_idx=6,
        num_targets=2,
    )

    # 创建优化器，用于模型参数的更新
    optimizer = optim.Adam(surf_model.parameters(), lr=kwd["lr"])

    # 定义损失函数，用于衡量模型预测值与真实值的差异
    loss_func = nn.MSELoss()

    # 返回包含模型配置和训练所需信息的字典
    return {
        "prefix": prefix,
        "train_model": surf_model,
        "dset": dset,
        "optimizer": optimizer,
        "loss_func": loss_func,
    }

In [3]:
kwd = {
    "dropout": 0.2,
    "cnn_feature_ratio": 0.5,
    "data_root_path": data_root_path,
    "data_csv_filename": "DataNormilized.csv",
    "lr": 0.001,
    "num_params": 3,
    "num_output": 2,
}
suffix = "input254_cv5_train10000"
model_info = define_surf_model(
    model_name="densenet",
    model_type="121",
    pretrain=False,
    suffix=suffix,
    set_type="test",
    kwd=kwd,
)
model_name = f"{model_info['prefix']}"
root_path = "../../../thesis/surfTopo/checkpoints"

best_checkpoint = torch.load(
    os.path.join(root_path, model_name, f"surf_{model_name}_best.ckpt")
)
# latest_checkpoint = torch.load(
#     os.path.join(root_path, model_name, f"surf_{model_name}_latest.ckpt")
# )

In [4]:
surf_model = model_info["train_model"]
surf_model.load_state_dict(best_checkpoint["model_state_dict"])
optimizer = model_info["optimizer"]
optimizer.load_state_dict(best_checkpoint["optimizer_state_dict"])
test_set = model_info["dset"]
loss_func = model_info["loss_func"]
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

In [5]:
for name, module in surf_model.named_modules():
    if isinstance(module, nn.ReLU):
        # print(name)
        setattr(module, "inplace", False)

In [6]:
from util.visual.torch.cnn.cam import GradCam

grad_cam = GradCam(model=surf_model, verbose=True)

Following target layer is available:
['conv0', 'norm0', 'relu0', 'pool0', 'denseblock1', 'transition1', 'denseblock2', 'transition2', 'denseblock3', 'transition3', 'denseblock4', 'norm5']


In [10]:
import matplotlib.pyplot as plt

input_kwds = {"cmap": "gist_rainbow"}

fdict = grad_cam.get_feature_layer_dict()
fig, axes = plt.subplots(1, len(fdict), figsize=(5 * len(fdict), 5))
for batch in test_loader:
    input_surf = batch[0][0].detach().numpy()
    for i, target_layer in enumerate(fdict):
        print(target_layer)

        cam, _ = grad_cam.generate_cam(batch, target_layer)
        axes[i].imshow(cam, **input_kwds, alpha=0.4)
    for ax in axes.ravel():
        ax.axis("off")

    plt.savefig("./fig/grad_cam_layers.png")
    plt.close()
    break

conv0
Target layer [conv0] found
norm0
Target layer [norm0] found
relu0
Target layer [relu0] found
pool0
Target layer [pool0] found
denseblock1
Target layer [denseblock1] found
transition1
Target layer [transition1] found
denseblock2
Target layer [denseblock2] found
transition2
Target layer [transition2] found
denseblock3
Target layer [denseblock3] found
transition3
Target layer [transition3] found
denseblock4
Target layer [denseblock4] found
norm5
Target layer [norm5] found


In [10]:
from util.visual.torch.cnn.cam import LayerCam

layer_cam = LayerCam(model=surf_model)
for batch in test_loader:
    cam, conv_out = layer_cam.generate_cam(batch, target_layer="conv0")
    break
input_kwds = {"cmap": "gist_rainbow"}
fig, axes = plt.subplots(2, 2, figsize=(20, 20))
axes[0, 0].imshow(input_surf[0], **input_kwds)
axes[0, 1].imshow(input_surf[1], **input_kwds)
axes[1, 0].imshow(input_surf[0] + input_surf[1], **input_kwds)
axes[1, 1].imshow(cam, **input_kwds, alpha=0.4)

for ax in axes.ravel():
    ax.axis("off")

plt.savefig("./fig/layer_cam.png")
plt.close()

In [11]:
from util.visual.torch.cnn.cam import ScoreCam

score_cam = ScoreCam(model=surf_model)
for batch in test_loader:
    cam, conv_out = score_cam.generate_cam(batch, target_layer="conv0")
    break
input_kwds = {"cmap": "gist_rainbow"}
fig, axes = plt.subplots(2, 2, figsize=(20, 20))
axes[0, 0].imshow(input_surf[0], **input_kwds)
axes[0, 1].imshow(input_surf[1], **input_kwds)
axes[1, 0].imshow(input_surf[0] + input_surf[1], **input_kwds)
axes[1, 1].imshow(cam, **input_kwds, alpha=0.4)

for ax in axes.ravel():
    ax.axis("off")

plt.savefig("./fig/score_cam.png")
plt.close()